In [1]:
import cv2
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import keras 
from keras.applications.resnet import ResNet152

Using TensorFlow backend.


In [2]:
data = pd.read_csv("COVID.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Columns: 150532 entries, Unnamed: 0 to labels
dtypes: int64(150530), object(2)
memory usage: 57.4+ MB


In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,0,1,2,3,4,5,6,...,150519,150520,150521,150522,150523,150524,150525,150526,150527,labels
0,0,0,nejmc2001573_f1a.jpeg,71,69,68,71,69,68,71,...,67,65,65,67,65,65,67,65,65,covid
1,1,1,nejmoa2001191_f4.jpeg,55,52,54,55,52,54,55,...,55,52,54,55,52,54,55,52,54,covid
2,2,2,lancet-case2b.jpg,61,60,64,70,68,72,78,...,72,71,73,71,70,72,92,91,92,covid
3,3,3,radiol.2020200490.fig3.jpeg,33,33,33,1,1,1,42,...,12,12,12,13,13,13,36,36,36,covid
4,4,4,nejmoa2001191_f5-PA.jpeg,59,56,58,59,56,58,59,...,59,56,58,59,56,58,70,67,69,covid


In [5]:
data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [6]:
data = data.set_index('Unnamed: 0.1.1')

In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,150519,150520,150521,150522,150523,150524,150525,150526,150527,labels
Unnamed: 0.1.1,,,,,,,,,,,,,,,,,,,,,
nejmc2001573_f1a.jpeg,71,69,68,71,69,68,71,69,68,72,...,67,65,65,67,65,65,67,65,65,covid
nejmoa2001191_f4.jpeg,55,52,54,55,52,54,55,52,54,55,...,55,52,54,55,52,54,55,52,54,covid
lancet-case2b.jpg,61,60,64,70,68,72,78,77,78,82,...,72,71,73,71,70,72,92,91,92,covid
radiol.2020200490.fig3.jpeg,33,33,33,1,1,1,42,42,42,52,...,12,12,12,13,13,13,36,36,36,covid
nejmoa2001191_f5-PA.jpeg,59,56,58,59,56,58,59,56,58,59,...,59,56,58,59,56,58,70,67,69,covid


In [8]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40, 150528), (10, 150528), (40,), (10,))

In [11]:
model = ResNet152(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [12]:
model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [13]:
from keras.layers import GlobalAveragePooling2D, Dense

In [14]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(2,activation='sigmoid')(x) 

In [15]:
model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________________________________________________________________________

In [16]:
from keras.models import Model

In [17]:
model=Model(inputs=model.input,outputs=preds)

In [18]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1_conv
3 conv1_bn
4 conv1_relu
5 pool1_pad
6 pool1_pool
7 conv2_block1_1_conv
8 conv2_block1_1_bn
9 conv2_block1_1_relu
10 conv2_block1_2_conv
11 conv2_block1_2_bn
12 conv2_block1_2_relu
13 conv2_block1_0_conv
14 conv2_block1_3_conv
15 conv2_block1_0_bn
16 conv2_block1_3_bn
17 conv2_block1_add
18 conv2_block1_out
19 conv2_block2_1_conv
20 conv2_block2_1_bn
21 conv2_block2_1_relu
22 conv2_block2_2_conv
23 conv2_block2_2_bn
24 conv2_block2_2_relu
25 conv2_block2_3_conv
26 conv2_block2_3_bn
27 conv2_block2_add
28 conv2_block2_out
29 conv2_block3_1_conv
30 conv2_block3_1_bn
31 conv2_block3_1_relu
32 conv2_block3_2_conv
33 conv2_block3_2_bn
34 conv2_block3_2_relu
35 conv2_block3_3_conv
36 conv2_block3_3_bn
37 conv2_block3_add
38 conv2_block3_out
39 conv3_block1_1_conv
40 conv3_block1_1_bn
41 conv3_block1_1_relu
42 conv3_block1_2_conv
43 conv3_block1_2_bn
44 conv3_block1_2_relu
45 conv3_block1_0_conv
46 conv3_block1_3_conv
47 conv3_block1_0_bn
48 conv3_block1_3_bn

In [19]:
for layer in model.layers:
    layer.trainable=False

In [20]:
model.compile(loss="categorical_crossentropy", optimizer='adam',
             metrics=['accuracy'])

In [21]:
from keras.utils import to_categorical

In [22]:
X_train = X_train.reshape(len(X_train), 224, 224, 3)

In [23]:
X_test = X_test.reshape(len(X_test), 224, 224, 3)

#y_train = to_categorical(y_train, 2)
#y_test  = to_categorical(y_test, 2)

In [24]:
X_train.shape

(40, 224, 224, 3)

In [25]:
type(X_train)

numpy.ndarray

In [26]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

In [27]:
y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)

In [28]:
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [30]:
history = model.fit(x=X_train, y=y_train, steps_per_epoch=200,epochs=10, validation_data=(X_test, y_test), 
                    validation_steps=10)

Train on 40 samples, validate on 10 samples
Epoch 1/10


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1_conv/convolution (defined at /home/saireddy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_28448]

Function call stack:
keras_scratch_graph
